In [ ]:
from tensorflow.keras.layers import Conv2D, concatenate
from tensorflow.keras.models import Model 
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import Input
import numpy as np
from PIL import Image
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import matplotlib.gridspec as gridspec

In [ ]:
def preproc_images(name, size):
    x_train = []
    y_train = []
    for file in (os.listdir(name)):
        img = Image.open(name + file).convert('RGB')
        x = img.resize((size[0]//2, size[1]//2))
        x = x.resize(size, Image.NEAREST)
        x = np.array(x)
        y = img.resize(size)
        y = np.array(y)
        
        x_train.append(x)
        y_train.append(y)
        
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    x_train = x_train / 255
    y_train = y_train / 255
    
    return x_train, y_train

In [ ]:
x_train, y_train = preproc_images("Train/", (128, 128))

In [ ]:
input = Input(shape = (128,128, 3))
conv1 = Conv2D(64, (3, 3), activation = 'relu', padding = 'same')(input)
conv2 = Conv2D(64, (3, 3), activation = 'relu', padding = 'same')(conv1)

level1 = Conv2D(64, (3, 3), activation='relu', padding = 'same')(conv2)
level2 = Conv2D(64, (3, 3), activation='relu', padding = 'same')(conv2)
level3 = Conv2D(64, (3, 3), activation='relu', padding = 'same')(conv2)

concat = concatenate([level1, level2, level3], axis = 3)

conv3 = Conv2D(32, (3, 3), activation = 'relu', padding = 'same')(concat)
out = Conv2D(3, (3, 3), activation = 'sigmoid', padding = 'same')(conv3)

model = Model(inputs = input, outputs = out)

print (model.summary())

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss',
                              factor = 0.2,
                              patience = 6,
                              verbose = 1,
                              min_delta = 0.0001)
filepath = "weights.hdf5"

checkpoint = ModelCheckpoint(filepath, 
                             monitor = 'val_loss', 
                             verbose = 1, 
                             save_best_only=True, 
                             mode = 'min')

model.compile(loss = 'mse', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
history = model.fit(x_train, y_train,validation_split = 0.2 , epochs = 100, batch_size = 4, callbacks = [checkpoint, reduce_lr])

In [ ]:
def test_img(name, size):
    x_test = []
    for file in (os.listdir(name)):
        img = Image.open(name + file).convert('RGB')
        img = img.resize((size[0]//2, size[1]//2))
        img = img.resize(size, Image.NEAREST)
        img = np.array(img)
        img = img / 255
        img = np.array([img])
        x_test.append(img) 
    return x_test

In [ ]:
testimages = test_img("Test/", (128, 128))

In [ ]:
model.load_weights('weights.hdf5')

In [ ]:
func, ax = plt.subplots(3, 2, sharex = 'col', sharey = 'row', gridspec_kw = {'hspace': 0, 'wspace': 0}, squeeze = True)
plt.subplots_adjust(wspace = None, hspace = None)
func.set_figheight(15)
func.set_figwidth(15)
a = 0
k = 0
for i in range(3):  
    prediction = model.predict(testimages[a])
    prediction = prediction[0] * 255
    prediction = prediction.astype(np.uint8)
    prediction = Image.fromarray(prediction)
    ax[i][k].axis("off")
    ax[i][k].imshow(testimages[a].reshape(128, 128, 3))
    ax[i][k + 1].axis("off")
    ax[i][k + 1].imshow(prediction)
    a = a + 1
plt.show(block = False)